In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.window import Window

spark= SparkSession \
       .builder \
       .appName("daily_practice") \
       .config('spark.ui.port', '4050') \
       .getOrCreate()


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,659 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,718 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-s

In [2]:
from google.colab import output
output.serve_kernel_port_as_window(4050, path ='/jobs/index.html')

Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

# 🚩 **Day 7 - 2025/05/02**

### Q1: Window Function for Moving Calculations
Calculate a moving average of stock prices for multiple companies. For each stock, show the current day's closing price along with the 3-day and 7-day moving averages.

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, row_number
from pyspark.sql.window import Window

# Sample stock price data
stock_data = spark.createDataFrame([
    ("AAPL", "2024-04-01", 175.20),
    ("AAPL", "2024-04-02", 177.50),
    ("AAPL", "2024-04-03", 176.80),
    ("AAPL", "2024-04-04", 179.30),
    ("AAPL", "2024-04-05", 180.25),
    ("AAPL", "2024-04-08", 182.15),
    ("AAPL", "2024-04-09", 181.45),
    ("AAPL", "2024-04-10", 183.75),
    ("MSFT", "2024-04-01", 410.50),
    ("MSFT", "2024-04-02", 412.80),
    ("MSFT", "2024-04-03", 408.25),
    ("MSFT", "2024-04-04", 415.75),
    ("MSFT", "2024-04-05", 417.90),
    ("MSFT", "2024-04-08", 420.10),
    ("MSFT", "2024-04-09", 419.25),
    ("MSFT", "2024-04-10", 422.50)
], ["ticker", "date", "close_price"])
```

Expected output:
```
+-------+----------+-----------+------------------+------------------+
|ticker |date      |close_price|moving_avg_3day   |moving_avg_7day   |
+-------+----------+-----------+------------------+------------------+
|AAPL   |2024-04-10|183.75     |182.45            |181.02            |
|AAPL   |2024-04-09|181.45     |181.28            |179.61            |
|AAPL   |2024-04-08|182.15     |180.88            |178.74            |
|...    |...       |...        |...               |...               |
|MSFT   |2024-04-10|422.50     |420.62            |416.68            |
|...    |...       |...        |...               |...               |
+-------+----------+-----------+------------------+------------------+
```


In [3]:
# current day's closing price along with the 3-day and 7-day moving averages.
from pyspark.sql.functions import col, avg, row_number
from pyspark.sql.window import Window

# Sample stock price data
stock_data = spark.createDataFrame([
    ("AAPL", "2024-04-01", 175.20),
    ("AAPL", "2024-04-02", 177.50),
    ("AAPL", "2024-04-03", 176.80),
    ("AAPL", "2024-04-04", 179.30),
    ("AAPL", "2024-04-05", 180.25),
    ("AAPL", "2024-04-08", 182.15),
    ("AAPL", "2024-04-09", 181.45),
    ("AAPL", "2024-04-10", 183.75),
    ("MSFT", "2024-04-01", 410.50),
    ("MSFT", "2024-04-02", 412.80),
    ("MSFT", "2024-04-03", 408.25),
    ("MSFT", "2024-04-04", 415.75),
    ("MSFT", "2024-04-05", 417.90),
    ("MSFT", "2024-04-08", 420.10),
    ("MSFT", "2024-04-09", 419.25),
    ("MSFT", "2024-04-10", 422.50)
], ["ticker", "date", "close_price"])

# last two prices and current price
window_3_day = Window.partitionBy("ticker").orderBy("date").rowsBetween(Window.currentRow - 2, Window.currentRow)

# last six prices and current price
window_7_day = Window.partitionBy("ticker").orderBy("date").rowsBetween(Window.currentRow - 6, Window.currentRow)


stock_data = stock_data.withColumn("date", F.to_date("date"))

moving_averages = stock_data.withColumn("moving_avg_3day", F.round(F.avg("close_price").over(window_3_day), 2))\
                            .withColumn("moving_avg_7day", F.round(F.avg("close_price").over(window_7_day), 2))\
                            .select("ticker", "date", "close_price", "moving_avg_3day", "moving_avg_7day").orderBy(F.col("ticker").asc(), F.col("date").desc())

moving_averages.show()


+------+----------+-----------+---------------+---------------+
|ticker|      date|close_price|moving_avg_3day|moving_avg_7day|
+------+----------+-----------+---------------+---------------+
|  AAPL|2024-04-10|     183.75|         182.45|         180.17|
|  AAPL|2024-04-09|     181.45|         181.28|         178.95|
|  AAPL|2024-04-08|     182.15|         180.57|         178.53|
|  AAPL|2024-04-05|     180.25|         178.78|         177.81|
|  AAPL|2024-04-04|      179.3|         177.87|          177.2|
|  AAPL|2024-04-03|      176.8|          176.5|          176.5|
|  AAPL|2024-04-02|      177.5|         176.35|         176.35|
|  AAPL|2024-04-01|      175.2|          175.2|          175.2|
|  MSFT|2024-04-10|      422.5|         420.62|         416.65|
|  MSFT|2024-04-09|     419.25|         419.08|         414.94|
|  MSFT|2024-04-08|      420.1|         417.92|         414.22|
|  MSFT|2024-04-05|      417.9|         413.97|         413.04|
|  MSFT|2024-04-04|     415.75|         

### Q2: Data Deduplication with Business Rules
Remove duplicate customer records based on complex business rules: if multiple records exist for the same customer, keep the most complete record with preference to more recent updates.

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, coalesce, sum, max, row_number
from pyspark.sql.window import Window

# Sample customer data with duplicates and varying completeness
customer_data = spark.createDataFrame([
    (1001, "John", "Smith", "john.smith@email.com", "123-456-7890", "123 Main St", "New York", "NY", "2024-01-15"),
    (1001, "John", "Smith", "john.smith@email.com", None, "123 Main St", "New York", "NY", "2024-02-20"),
    (1001, "John", "Smith", "john.s@email.com", "123-456-7890", "123 Main St", None, "NY", "2024-03-05"),
    (1002, "Jane", "Doe", "jane.doe@email.com", "987-654-3210", None, "Los Angeles", "CA", "2024-01-10"),
    (1002, "Jane", "Doe", "jane.doe@email.com", "987-654-3210", "456 Oak Ave", "Los Angeles", "CA", "2024-02-25"),
    (1003, "Michael", "Johnson", "michael.j@email.com", "555-123-4567", "789 Pine St", "Chicago", "IL", "2024-01-20"),
    (1003, "Michael", "Johnson", "michael.johnson@email.com", "555-123-4567", "789 Pine St", "Chicago", "IL", "2024-02-10")
], ["customer_id", "first_name", "last_name", "email", "phone", "address", "city", "state", "last_updated"])
```

Expected output:
```
+------------+----------+---------+----------------------+-------------+-------------+-------------+-----+------------+
|customer_id |first_name|last_name|email                 |phone        |address      |city         |state|last_updated|
+------------+----------+---------+----------------------+-------------+-------------+-------------+-----+------------+
|1001        |John      |Smith    |john.smith@email.com  |123-456-7890 |123 Main St  |New York     |NY   |2024-03-05  |
|1002        |Jane      |Doe      |jane.doe@email.com    |987-654-3210 |456 Oak Ave  |Los Angeles  |CA   |2024-02-25  |
|1003        |Michael   |Johnson   |michael.johnson@email.com|555-123-4567|789 Pine St|Chicago      |IL   |2024-02-10  |
+------------+----------+---------+----------------------+-------------+-------------+-------------+-----+------------+
```

In [4]:
from pyspark.sql.functions import col, count, when, coalesce, sum, max, row_number
from pyspark.sql.window import Window

# Sample customer data with duplicates and varying completeness
customer_data = spark.createDataFrame([
    (1001, "John", "Smith", "john.smith@email.com", "123-456-7890", "123 Main St", "New York", "NY", "2024-01-15"),
    (1001, "John", "Smith", "john.smith@email.com", None, "123 Main St", "New York", "NY", "2024-02-20"),
    (1001, "John", "Smith", "john.s@email.com", "123-456-7890", "123 Main St", None, "NY", "2024-03-05"),
    (1002, "Jane", "Doe", "jane.doe@email.com", "987-654-3210", None, "Los Angeles", "CA", "2024-01-10"),
    (1002, "Jane", "Doe", "jane.doe@email.com", "987-654-3210", "456 Oak Ave", "Los Angeles", "CA", "2024-02-25"),
    (1003, "Michael", "Johnson", "michael.j@email.com", "555-123-4567", "789 Pine St", "Chicago", "IL", "2024-01-20"),
    (1003, "Michael", "Johnson", "michael.johnson@email.com", "555-123-4567", "789 Pine St", "Chicago", "IL", "2024-02-10")
], ["customer_id", "first_name", "last_name", "email", "phone", "address", "city", "state", "last_updated"])

customer_data = customer_data.withColumn("last_updated", F.to_date("last_updated"))

customer_data.select("*").orderBy(F.col("customer_id").asc(), F.col("last_updated").desc()).show()


+-----------+----------+---------+--------------------+------------+-----------+-----------+-----+------------+
|customer_id|first_name|last_name|               email|       phone|    address|       city|state|last_updated|
+-----------+----------+---------+--------------------+------------+-----------+-----------+-----+------------+
|       1001|      John|    Smith|    john.s@email.com|123-456-7890|123 Main St|       NULL|   NY|  2024-03-05|
|       1001|      John|    Smith|john.smith@email.com|        NULL|123 Main St|   New York|   NY|  2024-02-20|
|       1001|      John|    Smith|john.smith@email.com|123-456-7890|123 Main St|   New York|   NY|  2024-01-15|
|       1002|      Jane|      Doe|  jane.doe@email.com|987-654-3210|456 Oak Ave|Los Angeles|   CA|  2024-02-25|
|       1002|      Jane|      Doe|  jane.doe@email.com|987-654-3210|       NULL|Los Angeles|   CA|  2024-01-10|
|       1003|   Michael|  Johnson|michael.johnson@e...|555-123-4567|789 Pine St|    Chicago|   IL|  2024

In [5]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Step 1: Calculate completeness score
df_with_score = customer_data.withColumn("completeness_score",
    (F.col("customer_id").isNotNull().cast("int") +
     F.col("first_name").isNotNull().cast("int") +
     F.col("last_name").isNotNull().cast("int") +
     F.col("email").isNotNull().cast("int") +
     F.col("phone").isNotNull().cast("int") +
     F.col("address").isNotNull().cast("int") +
     F.col("city").isNotNull().cast("int") +
     F.col("state").isNotNull().cast("int") +
     F.col("last_updated").isNotNull().cast("int"))
)

# Step 2: Define window spec - ensure last_updated is correctly ordered
window_spec = Window.partitionBy("customer_id")\
                   .orderBy(F.col("completeness_score").desc(),
                            F.to_date(F.col("last_updated"), "yyyy-MM-dd").desc())

# Step 3: Rank the records
df_ranked = df_with_score.withColumn("rank", F.row_number().over(window_spec))

# Step 4: Filter to keep only the top-ranked record
df_deduplicated = df_ranked.filter(F.col("rank") == 1).drop("completeness_score", "rank")

In [6]:
df_deduplicated.show()

+-----------+----------+---------+--------------------+------------+-----------+-----------+-----+------------+
|customer_id|first_name|last_name|               email|       phone|    address|       city|state|last_updated|
+-----------+----------+---------+--------------------+------------+-----------+-----------+-----+------------+
|       1001|      John|    Smith|john.smith@email.com|123-456-7890|123 Main St|   New York|   NY|  2024-01-15|
|       1002|      Jane|      Doe|  jane.doe@email.com|987-654-3210|456 Oak Ave|Los Angeles|   CA|  2024-02-25|
|       1003|   Michael|  Johnson|michael.johnson@e...|555-123-4567|789 Pine St|    Chicago|   IL|  2024-02-10|
+-----------+----------+---------+--------------------+------------+-----------+-----------+-----+------------+



### Q3: Dynamic Pivot and Unpivot
Create a solution that can dynamically pivot and unpivot data without hardcoding column names. Transform sales data from a normalized format to a crosstab view by product category and quarter, then transform it back.

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, lit, concat, collect_list, struct, first, avg
import pyspark.sql.functions as F

# Sample sales data in normalized format
sales_data = spark.createDataFrame([
    ("Electronics", "Q1", 1250000),
    ("Electronics", "Q2", 1380000),
    ("Electronics", "Q3", 1120000),
    ("Electronics", "Q4", 1680000),
    ("Clothing", "Q1", 850000),
    ("Clothing", "Q2", 940000),
    ("Clothing", "Q3", 1020000),
    ("Clothing", "Q4", 1350000),
    ("Home Goods", "Q1", 720000),
    ("Home Goods", "Q2", 680000),
    ("Home Goods", "Q3", 790000),
    ("Home Goods", "Q4", 950000),
    ("Sports", "Q1", 450000),
    ("Sports", "Q2", 520000),
    ("Sports", "Q3", 670000),
    ("Sports", "Q4", 540000)
], ["category", "quarter", "sales"])

# You need to create two transformations:
# 1. Pivoted data - categories as rows, quarters as columns
# 2. Unpivoted data - return to original format from the pivoted data
```

Expected output for pivoted data:
```
+-------------+--------+--------+--------+--------+
|category     |Q1_sales|Q2_sales|Q3_sales|Q4_sales|
+-------------+--------+--------+--------+--------+
|Electronics  |1250000 |1380000 |1120000 |1680000 |
|Clothing     |850000  |940000  |1020000 |1350000 |
|Home Goods   |720000  |680000  |790000  |950000  |
|Sports       |450000  |520000  |670000  |540000  |
+-------------+--------+--------+--------+--------+
```

In [7]:
from pyspark.sql.functions import col, expr, lit, concat, collect_list, struct, first, avg
import pyspark.sql.functions as F

# Sample sales data in normalized format
sales_data = spark.createDataFrame([
    ("Electronics", "Q1", 1250000),
    ("Electronics", "Q2", 1380000),
    ("Electronics", "Q3", 1120000),
    ("Electronics", "Q4", 1680000),
    ("Clothing", "Q1", 850000),
    ("Clothing", "Q2", 940000),
    ("Clothing", "Q3", 1020000),
    ("Clothing", "Q4", 1350000),
    ("Home Goods", "Q1", 720000),
    ("Home Goods", "Q2", 680000),
    ("Home Goods", "Q3", 790000),
    ("Home Goods", "Q4", 950000),
    ("Sports", "Q1", 450000),
    ("Sports", "Q2", 520000),
    ("Sports", "Q3", 670000),
    ("Sports", "Q4", 540000)
], ["category", "quarter", "sales"])

sales_data.show()

# 1. Pivoted data - categories as rows, quarters as columns
sales_data_pivoted = sales_data.groupBy("category").pivot("quarter").agg(F.sum("sales"))
sales_data_pivoted.show()

+-----------+-------+-------+
|   category|quarter|  sales|
+-----------+-------+-------+
|Electronics|     Q1|1250000|
|Electronics|     Q2|1380000|
|Electronics|     Q3|1120000|
|Electronics|     Q4|1680000|
|   Clothing|     Q1| 850000|
|   Clothing|     Q2| 940000|
|   Clothing|     Q3|1020000|
|   Clothing|     Q4|1350000|
| Home Goods|     Q1| 720000|
| Home Goods|     Q2| 680000|
| Home Goods|     Q3| 790000|
| Home Goods|     Q4| 950000|
|     Sports|     Q1| 450000|
|     Sports|     Q2| 520000|
|     Sports|     Q3| 670000|
|     Sports|     Q4| 540000|
+-----------+-------+-------+

+-----------+-------+-------+-------+-------+
|   category|     Q1|     Q2|     Q3|     Q4|
+-----------+-------+-------+-------+-------+
|     Sports| 450000| 520000| 670000| 540000|
|Electronics|1250000|1380000|1120000|1680000|
|   Clothing| 850000| 940000|1020000|1350000|
| Home Goods| 720000| 680000| 790000| 950000|
+-----------+-------+-------+-------+-------+



In [8]:
# 2. Unpivoted data - return to original format from the pivoted data
sales_data_pivoted.unpivot("category", ["Q1", "Q2", "Q3", "Q4"],"quarter", "sales").show()

+-----------+-------+-------+
|   category|quarter|  sales|
+-----------+-------+-------+
|     Sports|     Q1| 450000|
|     Sports|     Q2| 520000|
|     Sports|     Q3| 670000|
|     Sports|     Q4| 540000|
|Electronics|     Q1|1250000|
|Electronics|     Q2|1380000|
|Electronics|     Q3|1120000|
|Electronics|     Q4|1680000|
|   Clothing|     Q1| 850000|
|   Clothing|     Q2| 940000|
|   Clothing|     Q3|1020000|
|   Clothing|     Q4|1350000|
| Home Goods|     Q1| 720000|
| Home Goods|     Q2| 680000|
| Home Goods|     Q3| 790000|
| Home Goods|     Q4| 950000|
+-----------+-------+-------+



### Q4: Sessionization with Complex Rules
Implement a sessionization algorithm that groups user activity into sessions. A new session starts after 30 minutes of inactivity OR when a user switches device types, regardless of time.

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lag, when, sum, lit, coalesce, unix_timestamp, row_number
from pyspark.sql.window import Window

# Sample user activity data
user_logs = spark.createDataFrame([
    ("user1", "2024-04-01 10:15:20", "mobile", "/home"),
    ("user1", "2024-04-01 10:18:35", "mobile", "/products"),
    ("user1", "2024-04-01 10:25:42", "mobile", "/product/123"),
    ("user1", "2024-04-01 10:30:21", "desktop", "/product/123"),  # Device change
    ("user1", "2024-04-01 10:32:15", "desktop", "/cart"),
    ("user1", "2024-04-01 10:35:10", "desktop", "/checkout"),
    ("user1", "2024-04-01 11:20:30", "desktop", "/home"),  # Time gap
    ("user2", "2024-04-01 14:05:12", "tablet", "/home"),
    ("user2", "2024-04-01 14:08:25", "tablet", "/search"),
    ("user2", "2024-04-01 14:12:30", "tablet", "/product/456"),
    ("user2", "2024-04-01 14:45:12", "tablet", "/product/789"),  # Time gap
    ("user2", "2024-04-01 14:50:30", "mobile", "/product/789"),  # Device change
    ("user2", "2024-04-01 14:52:45", "mobile", "/cart")
], ["user_id", "timestamp", "device", "page"])
```

Expected output:
```
+-------+-------------------+-------+-------------+----------+--------------------+
|user_id|timestamp          |device |page         |session_id|session_duration_min|
+-------+-------------------+-------+-------------+----------+--------------------+
|user1  |2024-04-01 10:15:20|mobile |/home        |1         |10.37               |
|user1  |2024-04-01 10:18:35|mobile |/products    |1         |10.37               |
|user1  |2024-04-01 10:25:42|mobile |/product/123 |1         |10.37               |
|user1  |2024-04-01 10:30:21|desktop|/product/123 |2         |4.82                |
|user1  |2024-04-01 10:32:15|desktop|/cart        |2         |4.82                |
|user1  |2024-04-01 10:35:10|desktop|/checkout    |2         |4.82                |
|user1  |2024-04-01 11:20:30|desktop|/home        |3         |0.00                |
|user2  |2024-04-01 14:05:12|tablet |/home        |1         |7.30                |
|user2  |2024-04-01 14:08:25|tablet |/search      |1         |7.30                |
|user2  |2024-04-01 14:12:30|tablet |/product/456 |1         |7.30                |
|user2  |2024-04-01 14:45:12|tablet |/product/789 |2         |0.00                |
|user2  |2024-04-01 14:50:30|mobile |/product/789 |3         |2.25                |
|user2  |2024-04-01 14:52:45|mobile |/cart        |3         |2.25                |
+-------+-------------------+-------+-------------+----------+--------------------+
```

In [9]:
from pyspark.sql.functions import col, lag, when, sum, lit, coalesce, unix_timestamp, row_number
from pyspark.sql.window import Window

# Sample user activity data
user_logs = spark.createDataFrame([
    ("user1", "2024-04-01 10:15:20", "mobile", "/home"),
    ("user1", "2024-04-01 10:18:35", "mobile", "/products"),
    ("user1", "2024-04-01 10:25:42", "mobile", "/product/123"),
    ("user1", "2024-04-01 10:30:21", "desktop", "/product/123"),  # Device change
    ("user1", "2024-04-01 10:32:15", "desktop", "/cart"),
    ("user1", "2024-04-01 10:35:10", "desktop", "/checkout"),
    ("user1", "2024-04-01 11:20:30", "desktop", "/home"),  # Time gap
    ("user2", "2024-04-01 14:05:12", "tablet", "/home"),
    ("user2", "2024-04-01 14:08:25", "tablet", "/search"),
    ("user2", "2024-04-01 14:12:30", "tablet", "/product/456"),
    ("user2", "2024-04-01 14:45:12", "tablet", "/product/789"),  # Time gap
    ("user2", "2024-04-01 14:50:30", "mobile", "/product/789"),  # Device change
    ("user2", "2024-04-01 14:52:45", "mobile", "/cart")
], ["user_id", "timestamp", "device", "page"])

user_logs = user_logs.withColumn("timestamp", F.to_timestamp("timestamp"))

user_logs.show()

+-------+-------------------+-------+------------+
|user_id|          timestamp| device|        page|
+-------+-------------------+-------+------------+
|  user1|2024-04-01 10:15:20| mobile|       /home|
|  user1|2024-04-01 10:18:35| mobile|   /products|
|  user1|2024-04-01 10:25:42| mobile|/product/123|
|  user1|2024-04-01 10:30:21|desktop|/product/123|
|  user1|2024-04-01 10:32:15|desktop|       /cart|
|  user1|2024-04-01 10:35:10|desktop|   /checkout|
|  user1|2024-04-01 11:20:30|desktop|       /home|
|  user2|2024-04-01 14:05:12| tablet|       /home|
|  user2|2024-04-01 14:08:25| tablet|     /search|
|  user2|2024-04-01 14:12:30| tablet|/product/456|
|  user2|2024-04-01 14:45:12| tablet|/product/789|
|  user2|2024-04-01 14:50:30| mobile|/product/789|
|  user2|2024-04-01 14:52:45| mobile|       /cart|
+-------+-------------------+-------+------------+



In [16]:
# check for new session
window_user_id = Window.partitionBy("user_id").orderBy("timestamp")

sessions = user_logs.withColumn("prev_timestamp", F.lag("timestamp", 1).over(window_user_id))\
                    .withColumn("prev_device", F.lag("device", 1).over(window_user_id))\
                    .withColumn("time_gap", F.expr("(unix_timestamp(timestamp) - unix_timestamp(prev_timestamp))/60"))\
                    .withColumn("is_new_session", F.when((F.col("time_gap")>30)|(F.col("device")!=F.col("prev_device")), 1).otherwise(0))


# Create cumulative session ID by incrementing by 1 when there's a new session
sessions = sessions \
                .withColumn("session_id",
                sum("is_new_session").over(Window.partitionBy("user_id").orderBy("timestamp")) + 1)


# create a session and user based window for session duration calc
window_session = Window.partitionBy("user_id", "session_id").orderBy("timestamp")


sessions.withColumn("session_start", F.first("timestamp").over(window_session))\
        .withColumn("session_end", F.last("timestamp").over(window_session))\
        .withColumn("session_duration", F.round(F.expr("(unix_timestamp(session_end) - unix_timestamp(session_start))/60"), 2))\
        .select("user_id", "timestamp", "device", "page", "session_id", "session_duration").show()

+-------+-------------------+-------+------------+----------+----------------+
|user_id|          timestamp| device|        page|session_id|session_duration|
+-------+-------------------+-------+------------+----------+----------------+
|  user1|2024-04-01 10:15:20| mobile|       /home|         1|             0.0|
|  user1|2024-04-01 10:18:35| mobile|   /products|         1|            3.25|
|  user1|2024-04-01 10:25:42| mobile|/product/123|         1|           10.37|
|  user1|2024-04-01 10:30:21|desktop|/product/123|         2|             0.0|
|  user1|2024-04-01 10:32:15|desktop|       /cart|         2|             1.9|
|  user1|2024-04-01 10:35:10|desktop|   /checkout|         2|            4.82|
|  user1|2024-04-01 11:20:30|desktop|       /home|         3|             0.0|
|  user2|2024-04-01 14:05:12| tablet|       /home|         1|             0.0|
|  user2|2024-04-01 14:08:25| tablet|     /search|         1|            3.22|
|  user2|2024-04-01 14:12:30| tablet|/product/456|  

### Q5: Data Quality Monitoring
Develop a data quality monitoring framework that checks for various types of data quality issues including null values, out-of-range values, duplicate records, and pattern violations, then generates a comprehensive data quality report.

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, isnan, isnull, length, regexp_extract, min, max, countDistinct
import pyspark.sql.functions as F

# Sample dataset with various data quality issues
customer_orders = spark.createDataFrame([
    (1001, "John Smith", "jsmith@email.com", "2024-03-15", 120.50, "credit_card", "completed"),
    (1002, "Jane Doe", "jane.doe@email.com", "2024-03-16", 85.75, "paypal", "completed"),
    (1003, "Bob Johnson", None, "2024-03-16", 45.99, "credit_card", "completed"),
    (1004, "Mary Williams", "mw@invalid", "2024-03-17", 210.25, "credit_card", "completed"),
    (1005, "Steve Brown", "steve.brown@email.com", "2024-03-18", -15.99, "paypal", "failed"),
    (1006, "Emily Davis", "emily.davis@email.com", "2024-03-18", 75.50, "credit_card", "pending"),
    (1007, "David Miller", "david.miller@email.com", "2024-03-19", 0.0, "gift_card", "completed"),
    (1008, "Sarah Wilson", "sarah.wilson@email.com", "2024-33-19", 125.99, "credit_card", "completed"),
    (1009, "Michael Taylor", "michael.taylor@email.com", "2024-03-20", 50.75, "unknown", "completed"),
    (1010, "Jennifer Garcia", "jennifer.garcia@email.com", "2024-03-20", 95.25, "credit_card", "completed"),
    (1010, "Jennifer Garcia", "jennifer.garcia@email.com", "2024-03-20", 95.25, "credit_card", "completed")  # Duplicate
], ["order_id", "customer_name", "email", "order_date", "order_amount", "payment_method", "order_status"])

# Define data quality rules
rules = [
    {"type": "not_null", "column": "email", "description": "Email should not be null"},
    {"type": "format", "column": "email", "pattern": "^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}$", "description": "Email format invalid"},
    {"type": "range", "column": "order_amount", "min": 0.01, "description": "Order amount should be positive"},
    {"type": "valid_values", "column": "payment_method", "values": ["credit_card", "paypal", "gift_card"], "description": "Unknown payment method"},
    {"type": "date_format", "column": "order_date", "format": "yyyy-MM-dd", "description": "Invalid date format"},
    {"type": "duplicates", "columns": ["order_id"], "description": "Duplicate order ID"}
]
```

Expected output should be a data quality report with metrics for each rule:
```
+-------------------------+--------+--------+----------+--------------------+
|rule_description         |failures|total   |failure_pct|failing_examples   |
+-------------------------+--------+--------+----------+--------------------+
|Email should not be null |1       |11      |9.09      |[1003: Bob Johnson] |
|Email format invalid     |1       |11      |9.09      |[1004: mw@invalid]  |
|Order amount should be...|2       |11      |18.18     |[1005: -15.99, ...]|
|Unknown payment method   |1       |11      |9.09      |[1009: unknown]     |
|Invalid date format      |1       |11      |9.09      |[1008: 2024-33-19]  |
|Duplicate order ID       |1       |11      |9.09      |[1010]              |
+-------------------------+--------+--------+----------+--------------------+
```


In [31]:
from pyspark.sql.functions import col, count, when, isnan, isnull, length, regexp_extract, min, max, countDistinct
import pyspark.sql.functions as F

# Sample dataset with various data quality issues
customer_orders = spark.createDataFrame([
    (1001, "John Smith", "jsmith@email.com", "2024-03-15", 120.50, "credit_card", "completed"),
    (1002, "Jane Doe", "jane.doe@email.com", "2024-03-16", 85.75, "paypal", "completed"),
    (1003, "Bob Johnson", None, "2024-03-16", 45.99, "credit_card", "completed"),
    (1004, "Mary Williams", "mw@invalid", "2024-03-17", 210.25, "credit_card", "completed"),
    (1005, "Steve Brown", "steve.brown@email.com", "2024-03-18", -15.99, "paypal", "failed"),
    (1006, "Emily Davis", "emily.davis@email.com", "2024-03-18", 75.50, "credit_card", "pending"),
    (1007, "David Miller", "david.miller@email.com", "2024-03-19", 0.0, "gift_card", "completed"),
    (1008, "Sarah Wilson", "sarah.wilson@email.com", "2024-33-19", 125.99, "credit_card", "completed"),
    (1009, "Michael Taylor", "michael.taylor@email.com", "2024-03-20", 50.75, "unknown", "completed"),
    (1010, "Jennifer Garcia", "jennifer.garcia@email.com", "2024-03-20", 95.25, "credit_card", "completed"),
    (1010, "Jennifer Garcia", "jennifer.garcia@email.com", "2024-03-20", 95.25, "credit_card", "completed")  # Duplicate
], ["order_id", "customer_name", "email", "order_date", "order_amount", "payment_method", "order_status"])

customer_orders = customer_orders.withColumn("order_date", F.to_date("order_date"))

# Define data quality rules
rules = [
    {"type": "not_null", "column": "email", "description": "Email should not be null"},
    {"type": "format", "column": "email", "pattern": "^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}$", "description": "Email format invalid"},
    {"type": "range", "column": "order_amount", "min": 0.01, "description": "Order amount should be positive"},
    {"type": "valid_values", "column": "payment_method", "values": ["credit_card", "paypal", "gift_card"], "description": "Unknown payment method"},
    {"type": "date_format", "column": "order_date", "format": "yyyy-MM-dd", "description": "Invalid date format"},
    {"type": "duplicates", "columns": ["order_id"], "description": "Duplicate order ID"}
]

In [32]:
customer_orders.show()

+--------+---------------+--------------------+----------+------------+--------------+------------+
|order_id|  customer_name|               email|order_date|order_amount|payment_method|order_status|
+--------+---------------+--------------------+----------+------------+--------------+------------+
|    1001|     John Smith|    jsmith@email.com|2024-03-15|       120.5|   credit_card|   completed|
|    1002|       Jane Doe|  jane.doe@email.com|2024-03-16|       85.75|        paypal|   completed|
|    1003|    Bob Johnson|                NULL|2024-03-16|       45.99|   credit_card|   completed|
|    1004|  Mary Williams|          mw@invalid|2024-03-17|      210.25|   credit_card|   completed|
|    1005|    Steve Brown|steve.brown@email...|2024-03-18|      -15.99|        paypal|      failed|
|    1006|    Emily Davis|emily.davis@email...|2024-03-18|        75.5|   credit_card|     pending|
|    1007|   David Miller|david.miller@emai...|2024-03-19|         0.0|     gift_card|   completed|


In [36]:
customer_orders = customer_orders.withColumn("is_email_null", F.col("email").isNull())\
                                .withColumn("is_email_valid", F.coalesce(F.col("email").rlike("^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}$"), F.lit(False)))\
                                .withColumn("is_order_amount_valid", F.when(F.col("order_amount")<0.01, False).otherwise(True))\
                                .withColumn("is_payment_method_valid", F.col("payment_method").isin(["credit_card", "paypal", "gift_card"]))\
                                .withColumn("is_date_valid", F.when(F.to_date("order_date").isNull(), False).otherwise(True))\
                                .withColumn("is_duplicate", F.when(F.row_number().over(Window.partitionBy("order_id").orderBy("order_date"))>1, True).otherwise(False))


In [38]:
total_records = customer_orders.count()
report_rows = []

fail_df = customer_orders.filter(F.col("is_email_null") == True)
fail_count = fail_df.count()
examples = fail_df.select("order_id", "customer_name").limit(5).collect()

report_rows.append({
    "rule_description": "Email should not be null",
    "failures": fail_count,
    "total": total_records,
    "failure_pct": round((fail_count / total_records) * 100, 2),
    "failing_examples": [f"{row['order_id']}: {row['customer_name']}" for row in examples]
})

fail_df = customer_orders.filter(F.col("is_email_valid") == False)
fail_count = fail_df.count()
examples = fail_df.select("order_id", "email").limit(5).collect()

report_rows.append({
    "rule_description": "Email format invalid",
    "failures": fail_count,
    "total": total_records,
    "failure_pct": round((fail_count / total_records) * 100, 2),
    "failing_examples": [f"{row['order_id']}: {row['email']}" for row in examples]
})

fail_df = customer_orders.filter(F.col("is_order_amount_valid") == False)
fail_count = fail_df.count()
examples = fail_df.select("order_id", "order_amount").limit(5).collect()

report_rows.append({
    "rule_description": "Order amount should be positive",
    "failures": fail_count,
    "total": total_records,
    "failure_pct": round((fail_count / total_records) * 100, 2),
    "failing_examples": [f"{row['order_id']}: {row['order_amount']}" for row in examples]
})


fail_df = customer_orders.filter(F.col("is_payment_method_valid") == False)
fail_count = fail_df.count()
examples = fail_df.select("order_id", "payment_method").limit(5).collect()

report_rows.append({
    "rule_description": "Unknown payment method",
    "failures": fail_count,
    "total": total_records,
    "failure_pct": round((fail_count / total_records) * 100, 2),
    "failing_examples": [f"{row['order_id']}: {row['payment_method']}" for row in examples]
})

fail_df = customer_orders.filter(F.col("is_date_valid") == False)
fail_count = fail_df.count()
examples = fail_df.select("order_id", "order_date").limit(5).collect()

report_rows.append({
    "rule_description": "Invalid date format",
    "failures": fail_count,
    "total": total_records,
    "failure_pct": round((fail_count / total_records) * 100, 2),
    "failing_examples": [f"{row['order_id']}: {row['order_date']}" for row in examples]
})

fail_df = customer_orders.filter(F.col("is_duplicate") == True)
fail_count = fail_df.count()
examples = fail_df.select("order_id").distinct().limit(5).collect()

report_rows.append({
    "rule_description": "Duplicate order ID",
    "failures": fail_count,
    "total": total_records,
    "failure_pct": round((fail_count / total_records) * 100, 2),
    "failing_examples": [str(row['order_id']) for row in examples]
})


report_df = spark.createDataFrame(report_rows)
report_df.select("rule_description", "failures", "total", "failure_pct", "failing_examples").show(truncate=False)


+-------------------------------+--------+-----+-----------+------------------------------+
|rule_description               |failures|total|failure_pct|failing_examples              |
+-------------------------------+--------+-----+-----------+------------------------------+
|Email should not be null       |1       |11   |9.09       |[1003: Bob Johnson]           |
|Email format invalid           |2       |11   |18.18      |[1003: None, 1004: mw@invalid]|
|Order amount should be positive|2       |11   |18.18      |[1005: -15.99, 1007: 0.0]     |
|Unknown payment method         |1       |11   |9.09       |[1009: unknown]               |
|Invalid date format            |1       |11   |9.09       |[1008: None]                  |
|Duplicate order ID             |1       |11   |9.09       |[1010]                        |
+-------------------------------+--------+-----+-----------+------------------------------+



### Q6: Advanced ETL with Error Handling and Recovery
Implement a robust ETL process that handles various types of data errors gracefully, routing invalid records to error tables with appropriate error messages while successfully processing valid data.

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, to_date, regexp_extract, trim, length, udf
from pyspark.sql.types import StringType, StructType, StructField, IntegerType, DoubleType, BooleanType, DateType

# Sample source data with various issues
source_data = spark.createDataFrame([
    ("1001", "John Smith", "1985-04-15", "US", "12345", "100,000.50", "Y"),
    ("1002", "Jane Doe", "1990-08-22", "CA", "ABC123", "75000.25", "N"),
    ("ABC", "Bob Johnson", "1975/12/30", "UK", "SW1A 1AA", "82,500.75", "Y"),
    ("1004", "Mary Williams", "2005-02-18", "US", "90210", "-25000.00", "Y"),
    ("1005", "Steve Brown", "1982:07:11", "AU", "2000", "invalid", "Z"),
    ("1006", "Emily Davis", "2000-13-45", "US", "60610", "65000.30", "N"),
    ("1007", "", "1995-05-20", "DE", "10115", "120000.00", "Y"),
    ("1008", "David Miller", "1988-03-10", "FR", "75001", "95,500.80", "X"),
    ("1009", "Sarah Wilson", "1979-11-30", None, "Unknown", "82000.45", "N"),
    ("1010", "Michael Taylor", "future", "US", "20500", "110,000.00", "Y")
], ["id", "name", "birth_date", "country", "postal_code", "salary", "active"])

# Target schema with proper data types
target_schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("name", StringType(), False),
    StructField("birth_date", DateType(), True),
    StructField("country", StringType(), True),
    StructField("postal_code", StringType(), True),
    StructField("salary", DoubleType(), True),
    StructField("active", BooleanType(), True)
])
```

Your solution should:
1. Transform and validate each field according to the target schema requirements
2. Create three dataframes: successfully processed records, records with errors, and an error log with detailed information
3. Include proper error messages for each type of validation failure
4. Implement a way to recover and fix simple errors automatically

In [41]:
from pyspark.sql.functions import col, when, lit, to_date, regexp_extract, trim, length, udf, regexp_replace
from pyspark.sql.types import StringType, StructType, StructField, IntegerType, DoubleType, BooleanType, DateType

# Sample source data with various issues
source_data = spark.createDataFrame([
    ("1001", "John Smith", "1985-04-15", "US", "12345", "100,000.50", "Y"),
    ("1002", "Jane Doe", "1990-08-22", "CA", "ABC123", "75000.25", "N"),
    ("ABC", "Bob Johnson", "1975/12/30", "UK", "SW1A 1AA", "82,500.75", "Y"),
    ("1004", "Mary Williams", "2005-02-18", "US", "90210", "-25000.00", "Y"),
    ("1005", "Steve Brown", "1982:07:11", "AU", "2000", "invalid", "Z"),
    ("1006", "Emily Davis", "2000-13-45", "US", "60610", "65000.30", "N"),
    ("1007", "", "1995-05-20", "DE", "10115", "120000.00", "Y"),
    ("1008", "David Miller", "1988-03-10", "FR", "75001", "95,500.80", "X"),
    ("1009", "Sarah Wilson", "1979-11-30", None, "Unknown", "82000.45", "N"),
    ("1010", "Michael Taylor", "future", "US", "20500", "110,000.00", "Y")
], ["id", "name", "birth_date", "country", "postal_code", "salary", "active"])

# Target schema with proper data types
target_schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("name", StringType(), False),
    StructField("birth_date", DateType(), True),
    StructField("country", StringType(), True),
    StructField("postal_code", StringType(), True),
    StructField("salary", DoubleType(), True),
    StructField("active", BooleanType(), True)
])

In [42]:
# Step 1: Validate and clean up the `salary` field (numeric issues)
df = source_data.withColumn(
    "salary_cleaned",
    when(col("salary").rlike("^[0-9,]+(\.[0-9]+)?$"), regexp_replace(col("salary"), ",", "").cast(DoubleType()))
    .otherwise(lit(None))
)

# Step 2: Handle invalid salary errors
df = df.withColumn(
    "salary_error",
    when(col("salary_cleaned").isNull() & col("salary").isNotNull(), "Invalid salary value")
    .otherwise(None)
)

# Step 3: Handle the `birth_date` field
df = df.withColumn("parsed_birth_date", to_date("birth_date", "yyyy-MM-dd"))

# Step 4: Identify and log invalid `birth_date` format errors
df = df.withColumn(
    "birth_date_error",
    when(col("parsed_birth_date").isNull() & col("birth_date").isNotNull(), "Invalid date format").otherwise(None)
)

# Step 5: Handle `country` field - checking for missing or invalid country codes
df = df.withColumn(
    "country_error",
    when(col("country").isNull() | (trim(col("country")) == ""), "Missing country").otherwise(None)
)

# Step 6: Handle `postal_code` field - checking for invalid postal codes (e.g., non-numeric or malformed)
df = df.withColumn(
    "postal_code_error",
    when(col("postal_code").rlike("^[A-Za-z0-9]+$"), lit(None)).otherwise("Invalid postal code")
)

# Step 7: Handle the `active` field
df = df.withColumn(
    "active_error",
    when(~col("active").isin("Y", "N"), "Invalid active status").otherwise(None)
)

# Step 8: Split into valid and invalid data
valid_data = df.filter(
    (col("salary_cleaned").isNotNull()) &
    (col("parsed_birth_date").isNotNull()) &
    (col("country_error").isNull()) &
    (col("postal_code_error").isNull()) &
    (col("active_error").isNull())
)

# Invalid records are those that have any error
invalid_data = df.filter(
    (col("salary_error").isNotNull()) |
    (col("birth_date_error").isNotNull()) |
    (col("country_error").isNotNull()) |
    (col("postal_code_error").isNotNull()) |
    (col("active_error").isNotNull())
)

# Step 9: Create error log with detailed messages
error_log = invalid_data.select(
    "id", "salary_error", "birth_date_error", "country_error", "postal_code_error", "active_error"
).withColumn(
    "error_messages",
    when(col("salary_error").isNotNull(), col("salary_error"))
    .otherwise(lit("")) +
    when(col("birth_date_error").isNotNull(), lit(" | ") + col("birth_date_error"))
    .otherwise(lit("")) +
    when(col("country_error").isNotNull(), lit(" | ") + col("country_error"))
    .otherwise(lit("")) +
    when(col("postal_code_error").isNotNull(), lit(" | ") + col("postal_code_error"))
    .otherwise(lit("")) +
    when(col("active_error").isNotNull(), lit(" | ") + col("active_error"))
    .otherwise(lit(""))
)

# Step 10: Show results
valid_data.show(truncate=False)
invalid_data.show(truncate=False)
error_log.show(truncate=False)


+----+----------+----------+-------+-----------+----------+------+--------------+------------+-----------------+----------------+-------------+-----------------+------------+
|id  |name      |birth_date|country|postal_code|salary    |active|salary_cleaned|salary_error|parsed_birth_date|birth_date_error|country_error|postal_code_error|active_error|
+----+----------+----------+-------+-----------+----------+------+--------------+------------+-----------------+----------------+-------------+-----------------+------------+
|1001|John Smith|1985-04-15|US     |12345      |100,000.50|Y     |100000.5      |NULL        |1985-04-15       |NULL            |NULL         |NULL             |NULL        |
|1002|Jane Doe  |1990-08-22|CA     |ABC123     |75000.25  |N     |75000.25      |NULL        |1990-08-22       |NULL            |NULL         |NULL             |NULL        |
|1007|          |1995-05-20|DE     |10115      |120000.00 |Y     |120000.0      |NULL        |1995-05-20       |NULL         

### Q7: Customer Churn Analysis and Prediction Features
Create features for a customer churn prediction model by analyzing customer behavior patterns in the months leading up to either churning or remaining active.

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, datediff, avg, sum, count, stddev, when, max, min, lag, dense_rank
from pyspark.sql.window import Window

# Customer subscription data
subscriptions = spark.createDataFrame([
    (101, "active", "2023-01-15", None),
    (102, "churned", "2023-02-10", "2023-11-15"),
    (103, "active", "2023-03-20", None),
    (104, "churned", "2023-01-05", "2023-08-20"),
    (105, "active", "2023-04-12", None),
    (106, "churned", "2023-02-28", "2023-06-05"),
    (107, "active", "2023-05-15", None),
    (108, "churned", "2023-03-10", "2023-12-22"),
    (109, "active", "2023-01-20", None),
    (110, "churned", "2023-04-05", "2023-10-10")
], ["customer_id", "status", "start_date", "end_date"])

# Customer monthly activity data
customer_activity = spark.createDataFrame([
    # Customer 101 - Active
    (101, "2023-05", 5, 450.00, 8, 2),
    (101, "2023-06", 4, 380.00, 6, 1),
    (101, "2023-07", 3, 310.00, 5, 0),
    (101, "2023-08", 5, 420.00, 7, 1),
    (101, "2023-09", 4, 350.00, 6, 0),
    (101, "2023-10", 6, 520.00, 9, 2),
    (101, "2023-11", 5, 480.00, 7, 1),
    (101, "2023-12", 4, 420.00, 8, 0),
    
    # Customer 102 - Churned
    (102, "2023-05", 5, 400.00, 7, 1),
    (102, "2023-06", 3, 250.00, 5, 0),
    (102, "2023-07", 2, 180.00, 4, 2),
    (102, "2023-08", 2, 150.00, 3, 1),
    (102, "2023-09", 1, 120.00, 2, 2),
    (102, "2023-10", 1, 100.00, 1, 3),
    (102, "2023-11", 0, 0.00, 0, 1),
    
    # More customers with similar patterns...
    (103, "2023-06", 4, 350.00, 6, 0),
    (103, "2023-07", 5, 420.00, 8, 1),
    (103, "2023-08", 6, 510.00, 9, 0),
    (103, "2023-09", 5, 480.00, 7, 1),
    (103, "2023-10", 7, 580.00, 10, 0),
    (103, "2023-11", 6, 520.00, 8, 0),
    (103, "2023-12", 5, 490.00, 9, 1),
    
    (104, "2023-04", 4, 350.00, 6, 1),
    (104, "2023-05", 3, 280.00, 5, 2),
    (104, "2023-06", 2, 220.00, 4, 1),
    (104, "2023-07", 1, 150.00, 3, 3),
    (104, "2023-08", 0, 0.00, 0, 2)
    
    # Additional customers would be included here...
], ["customer_id", "month", "order_count", "spend_amount", "site_visits", "support_tickets"])

# Customer demographic data
customer_demographics = spark.createDataFrame([
    (101, "Individual", "Premium", 12),
    (102, "Business", "Basic", 9),
    (103, "Individual", "Premium", 10),
    (104, "Business", "Standard", 7),
    (105, "Individual", "Basic", 5),
    (106, "Business", "Premium", 4),
    (107, "Individual", "Standard", 8),
    (108, "Business", "Basic", 11),
    (109, "Individual", "Premium", 14),
    (110, "Business", "Standard", 6)
], ["customer_id", "customer_type", "subscription_tier", "tenure_months"])
```

Your solution should create features including:
1. Activity trends in the 3 months before churning or the most recent 3 months for active customers
2. Spending decline rate
3. Customer engagement metrics
4. Support ticket frequency
5. Early warning indicators

In [45]:
from pyspark.sql.functions import col, datediff, avg, sum, count, stddev, when, max, min, lag, dense_rank
from pyspark.sql.window import Window

# Customer subscription data
subscriptions = spark.createDataFrame([
    (101, "active", "2023-01-15", None),
    (102, "churned", "2023-02-10", "2023-11-15"),
    (103, "active", "2023-03-20", None),
    (104, "churned", "2023-01-05", "2023-08-20"),
    (105, "active", "2023-04-12", None),
    (106, "churned", "2023-02-28", "2023-06-05"),
    (107, "active", "2023-05-15", None),
    (108, "churned", "2023-03-10", "2023-12-22"),
    (109, "active", "2023-01-20", None),
    (110, "churned", "2023-04-05", "2023-10-10")
], ["customer_id", "status", "start_date", "end_date"])

# Customer monthly activity data
customer_activity = spark.createDataFrame([
    # Customer 101 - Active
    (101, "2023-05", 5, 450.00, 8, 2),
    (101, "2023-06", 4, 380.00, 6, 1),
    (101, "2023-07", 3, 310.00, 5, 0),
    (101, "2023-08", 5, 420.00, 7, 1),
    (101, "2023-09", 4, 350.00, 6, 0),
    (101, "2023-10", 6, 520.00, 9, 2),
    (101, "2023-11", 5, 480.00, 7, 1),
    (101, "2023-12", 4, 420.00, 8, 0),

    # Customer 102 - Churned
    (102, "2023-05", 5, 400.00, 7, 1),
    (102, "2023-06", 3, 250.00, 5, 0),
    (102, "2023-07", 2, 180.00, 4, 2),
    (102, "2023-08", 2, 150.00, 3, 1),
    (102, "2023-09", 1, 120.00, 2, 2),
    (102, "2023-10", 1, 100.00, 1, 3),
    (102, "2023-11", 0, 0.00, 0, 1),

    # More customers with similar patterns...
    (103, "2023-06", 4, 350.00, 6, 0),
    (103, "2023-07", 5, 420.00, 8, 1),
    (103, "2023-08", 6, 510.00, 9, 0),
    (103, "2023-09", 5, 480.00, 7, 1),
    (103, "2023-10", 7, 580.00, 10, 0),
    (103, "2023-11", 6, 520.00, 8, 0),
    (103, "2023-12", 5, 490.00, 9, 1),

    (104, "2023-04", 4, 350.00, 6, 1),
    (104, "2023-05", 3, 280.00, 5, 2),
    (104, "2023-06", 2, 220.00, 4, 1),
    (104, "2023-07", 1, 150.00, 3, 3),
    (104, "2023-08", 0, 0.00, 0, 2)

    # Additional customers would be included here...
], ["customer_id", "month", "order_count", "spend_amount", "site_visits", "support_tickets"])

# Customer demographic data
customer_demographics = spark.createDataFrame([
    (101, "Individual", "Premium", 12),
    (102, "Business", "Basic", 9),
    (103, "Individual", "Premium", 10),
    (104, "Business", "Standard", 7),
    (105, "Individual", "Basic", 5),
    (106, "Business", "Premium", 4),
    (107, "Individual", "Standard", 8),
    (108, "Business", "Basic", 11),
    (109, "Individual", "Premium", 14),
    (110, "Business", "Standard", 6)
], ["customer_id", "customer_type", "subscription_tier", "tenure_months"])

In [55]:
# Activity trends in the 3 months before churning or the most recent 3 months for active customers
# activity trend can be captured by the number of orders, amount spent, site visits, tickets raised
churned_users = subscriptions.filter(F.col("status")=="churned").join(customer_activity, on = ["customer_id"])

active_users = subscriptions.filter(F.col("status")=="active").join(customer_activity, on = ["customer_id"])

# add reference month
churned_users = churned_users.withColumn("reference_month", F.date_format("end_date", "yyyy-MM"))
active_users = active_users.groupBy("customer_id").agg(F.max(F.to_date("month", "yyyy-MM")).alias("reference_month"))\
                            .join(active_users, on = ["customer_id"])

churned_users = churned_users.withColumn("month_date", F.to_date("month", "yyyy-MM"))\
                             .withColumn("reference_date", F.to_date(F.col("reference_month"), "yyyy-MM"))

active_users = active_users.withColumn("month_date", F.to_date("month", "yyyy-MM"))\
                           .withColumn("reference_date", F.to_date(F.col("reference_month"), "yyyy-MM"))


# filter for last 3 months
churned_users_last3_months = churned_users.withColumn(
    "months_between", F.abs(F.months_between("month_date", "reference_date"))
).filter(F.col("months_between") <= 3)

active_users_last3_months = active_users.withColumn(
    "months_between", F.abs(F.months_between("month_date", "reference_date"))
).filter(F.col("months_between") <= 3)


In [56]:
agg_metrics_active = active_users_last3_months.groupBy("customer_id").agg(
    F.avg("order_count").alias("avg_order_count"),
    F.avg("spend_amount").alias("avg_spend"),
    F.avg("site_visits").alias("avg_site_visits"),
    F.avg("support_tickets").alias("avg_support_tickets")
)
agg_metrics_active.show()

agg_metrics_churned = churned_users_last3_months.groupBy("customer_id").agg(
    F.avg("order_count").alias("avg_order_count"),
    F.avg("spend_amount").alias("avg_spend"),
    F.avg("site_visits").alias("avg_site_visits"),
    F.avg("support_tickets").alias("avg_support_tickets")
)
agg_metrics_churned.show()

+-----------+---------------+---------+---------------+-------------------+
|customer_id|avg_order_count|avg_spend|avg_site_visits|avg_support_tickets|
+-----------+---------------+---------+---------------+-------------------+
|        101|           4.75|    442.5|            7.5|               0.75|
|        103|           5.75|    517.5|            8.5|                0.5|
+-----------+---------------+---------+---------------+-------------------+

+-----------+---------------+---------+---------------+-------------------+
|customer_id|avg_order_count|avg_spend|avg_site_visits|avg_support_tickets|
+-----------+---------------+---------+---------------+-------------------+
|        102|            1.0|     92.5|            1.5|               1.75|
|        104|            1.5|    162.5|            3.0|                2.0|
+-----------+---------------+---------+---------------+-------------------+



In [59]:
window_spec = Window.partitionBy("customer_id").orderBy("month")

active_trend_df = active_users_last3_months.withColumn("prev_order_count", F.lag("order_count").over(window_spec)) \
    .withColumn("order_count_diff", F.col("order_count") - F.col("prev_order_count")) \
    .filter(F.col("order_count_diff").isNotNull()) \
    .groupBy("customer_id").agg(F.avg("order_count_diff").alias("avg_order_count_diff"))

active_trend_df.show()


churned_trend_df = churned_users_last3_months.withColumn("prev_order_count", F.lag("order_count").over(window_spec)) \
    .withColumn("order_count_diff", F.col("order_count") - F.col("prev_order_count")) \
    .filter(F.col("order_count_diff").isNotNull()) \
    .groupBy("customer_id").agg(F.avg("order_count_diff").alias("avg_order_count_diff"))

churned_trend_df.show()

+-----------+--------------------+
|customer_id|avg_order_count_diff|
+-----------+--------------------+
|        101|                 0.0|
|        103|                 0.0|
+-----------+--------------------+

+-----------+--------------------+
|customer_id|avg_order_count_diff|
+-----------+--------------------+
|        102| -0.6666666666666666|
|        104|                -1.0|
+-----------+--------------------+



In [60]:
# Join all features together
features_active = agg_metrics_active.join(active_trend_df, on="customer_id")
features_churned = agg_metrics_churned.join(churned_trend_df, on="customer_id")

# Show results
features_active.show()
features_churned.show()

+-----------+---------------+---------+---------------+-------------------+--------------------+
|customer_id|avg_order_count|avg_spend|avg_site_visits|avg_support_tickets|avg_order_count_diff|
+-----------+---------------+---------+---------------+-------------------+--------------------+
|        101|           4.75|    442.5|            7.5|               0.75|                 0.0|
|        103|           5.75|    517.5|            8.5|                0.5|                 0.0|
+-----------+---------------+---------+---------------+-------------------+--------------------+

+-----------+---------------+---------+---------------+-------------------+--------------------+
|customer_id|avg_order_count|avg_spend|avg_site_visits|avg_support_tickets|avg_order_count_diff|
+-----------+---------------+---------+---------------+-------------------+--------------------+
|        102|            1.0|     92.5|            1.5|               1.75| -0.6666666666666666|
|        104|            1.5|

In [44]:

# Spending decline rate
# Customer engagement metrics
# Support ticket frequency
# Early warning indicators

### Q8: Anomaly Detection in Time Series Data
Develop an anomaly detection system that identifies unusual patterns in time series data using multiple statistical methods: Z-score, modified Z-score, and IQR (Interquartile Range).

```python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, stddev, abs, median, percentile_approx, lit, when
from pyspark.sql.window import Window

# Sample sensor data with anomalies
sensor_data = spark.createDataFrame([
    ("sensor1", "2024-04-01 00:00:00", 25.2),
    ("sensor1", "2024-04-01 01:00:00", 24.8),
    ("sensor1", "2024-04-01 02:00:00", 24.5),
    ("sensor1", "2024-04-01 03:00:00", 24.3),
    ("sensor1", "2024-04-01 04:00:00", 24.1),
    ("sensor1", "2024-04-01 05:00:00", 24.0),
    ("sensor1", "2024-04-01 06:00:00", 24.2),
    ("sensor1", "2024-04-01 07:00:00", 24.6),
    ("sensor1", "2024-04-01 08:00:00", 25.0),
    ("sensor1", "2024-04-01 09:00:00", 25.5),
    ("sensor1", "2024-04-01 10:00:00", 35.8),  # Anomaly
    ("sensor1", "2024-04-01 11:00:00", 26.5),
    ("sensor1", "2024-04-01 12:00:00", 26.8),
    ("sensor1", "2024-04-01 13:00:00", 27.1),
    ("sensor1", "2024-04-01 14:00:00", 27.3),
    ("sensor1", "2024-04-01 15:00:00", 17.2),  # Anomaly
    ("sensor1", "2024-04-01 16:00:00", 27.5),
    ("sensor1", "2024-04-01 17:00:00", 27.2),
    ("sensor1", "2024-04-01 18:00:00", 26.8),
    ("sensor1", "2024-04-01 19:00:00", 26.5),
    ("sensor1", "2024-04-01 20:00:00", 26.2),
    ("sensor1", "2024-04-01 21:00:00", 25.8),
    ("sensor1", "2024-04-01 22:00:00", 25.5),
    ("sensor1", "2024-04-01 23:00:00", 25.2),
    
    ("sensor2", "2024-04-01 00:00:00", 80.5),
    ("sensor2", "2024-04-01 01:00:00", 81.2),
    ("sensor2", "2024-04-01 02:00:00", 80.8),
    ("sensor2", "2024-04-01 03:00:00", 81.0),
    ("sensor2", "2024-04-01 04:00:00", 80.7),
    ("sensor2", "2024-04-01 05:00:00", 80.9),
    ("sensor2", "2024-04-01 06:00:00", 81.5),
    ("sensor2", "2024-04-01 07:00:00", 82.0),
    ("sensor2", "2024-04-01 08:00:00", 82.5),
    ("sensor2", "2024-04-01 09:00:00", 50.2),  # Anomaly
    ("sensor2", "2024-04-01 10:00:00", 83.5),
    ("sensor2", "2024-04-01 11:00:00", 84.0),
    ("sensor2", "2024-04-01 12:00:00", 120.8), # Anomaly
    ("sensor2", "2024-04-01 13:00:00", 84.5),
    ("sensor2", "2024-04-01 14:00:00", 85.0),
    ("sensor2", "2024-04-01 15:00:00", 84.8),
    ("sensor2", "2024-04-01 16:00:00", 84.5),
    ("sensor2", "2024-04-01 17:00:00", 84.2),
    ("sensor2", "2024-04-01 18:00:00", 83.8),
    ("sensor2", "2024-04-01 19:00:00", 83.5),
    ("sensor2", "2024-04-01 20:00:00", 83.0),
    ("sensor2", "2024-04-01 21:00:00", 82.5),
    ("sensor2", "2024-04-01 22:00:00", 82.0),
    ("sensor2", "2024-04-01 23:00:00", 81.5)
], ["sensor_id", "timestamp", "reading"])
```

Your solution should:
1. Implement three anomaly detection methods: Z-score, modified Z-score, and IQR
2. Compare the methods and identify which anomalies are detected by each method
3. Calculate an anomaly score for each data point
4. Visualize the anomalies in a meaningful way
